# SET UP

In [ ]:
%run ../CONNECTIONS/setup.ipynb

In [ ]:
%run Serology_All_Functions.ipynb

In [ ]:
# all files availables in the project folder
import glob

# Optional: View all the excel files in INPUT dir exported from GA
location = '../DATA/DSC2'
fileset = [file for file in glob.glob(location + "**/***.*", recursive=True)]
n = 0
for file in fileset:
    #print()
    print(n, file)
    n+=1

In [ ]:
# To save files with today's date/timestamp
today = str(datetime.today().strftime('%Y-%m-%d'))
time = str(datetime.now().time())[:8]
now = (today+'_'+time).replace(':','.')
now

In [ ]:
data_directory = '../DATA\\DSC2\\'

## Load needed Inputs

In [ ]:
## pull list of blood tests from dictionnary at 
# https://docs.google.com/spreadsheets/d/16TVEwQPYehExogaC-bFfZX3aBTKHFcdO1AeHUbnHOag/edit#gid=0

blood_tests = pd.read_csv('../DATA/common_inputs/blood_bio_tests.csv').drop_duplicates().iloc[:,1:]
blood_tests

In [ ]:
blood_tests2 = tuple(blood_tests.test)
#blood_tests

## Items to exclude from SQL Query

'*Specimens from AI/AN participants and participants who withdrew before the first plate map is completed will be excluded from testing*

In [ ]:
# All participants who have withdrawn
withdrawn = pd.read_sql('''
        SELECT distinct participant_id, biobank_id, withdrawal_status
        from participant_summary 
        where withdrawal_status =2

''', cnx)
withdrawn.head() 

In [ ]:
# All participants who have at least one AIAN race chosen in code table
atleast_oneAIAN_race = pd.read_sql('''
        SELECT distinct ps.participant_id, biobank_id, short_value as race_c
        from participant_summary ps
        left join participant_race_answers p USING(participant_id)
        left join code c USING(code_id)
        where short_value = 'WhatRaceEthnicity_AIAN'

''', cnx)
atleast_oneAIAN_race

In [ ]:
atleast_oneAIAN_race.participant_id.nunique()

In [ ]:

# onlyAIAN_race = pd.read_sql('''
#         SELECT count(distinct ps.participant_id)
#         from participant_summary ps
#         where race = 3
# ''', cnx)
# onlyAIAN_race

In [ ]:
PIDS_atleast_oneAIAN_race= tuple(atleast_oneAIAN_race.participant_id)
len(atleast_oneAIAN_race.participant_id.drop_duplicates())

--------------------------------------------------------------------------------------------

In [ ]:
wd_aian = tuple(set(list(withdrawn.participant_id) + list(atleast_oneAIAN_race.participant_id)))
len(wd_aian)

In [ ]:
# We also need to exclude fake, ghosts and tests pids  - USE DIRECTLY IN SQL QUERY
notfakes = '''p.is_ghost_id IS NULL
    AND (ps.email NOT LIKE '%@example.com' or ps.email IS NULL)
    AND p.hpo_id != 21
    #AND (ps.withdrawal_status = 1 or ps.withdrawal_status is null) ## excludes withdrawn too
    '''


In [ ]:
#NEW 6/11 To track the number of AI/AN and withdrawn per batch -- this is a proxy since we do not have them per batch
# This is all the AI/AN that meet the condition sin the Master List belo and would have been included in the query
AIAN_WD = pd.read_sql('''

    SELECT distinct participant_id, p.withdrawal_status, bos.collected as DateBloodSampleCollected

    FROM participant p
    INNER JOIN participant_summary ps USING(participant_id)
    INNER JOIN biobank_order bo USING(participant_id)
    INNER JOIN biobank_ordered_sample bos on bo.biobank_order_id = bos.order_id
    INNER JOIN biobank_order_identifier boi on boi.biobank_order_id = bo.biobank_order_id
    INNER JOIN biobank_stored_sample bss on bss.biobank_order_identifier = boi.value
    
    WHERE {0} #exclude withdrawn, fake, tests
    AND bos.collected > '2020-01-01 00:00:00'  # need to change this to '2020-01-01 00:00:00'per Mine's email on may 21, 2020
                                                #to avoid running the whole query again, the 2020 date constraint was adjust in 2.3
    AND bss.status = 1
    AND lower(bos.test) IN {1}
    
   ## group by 1, bos.test, bos.collected, bss.confirmed, bss.status
    
    '''.format(notfakes, blood_tests2), cnx)


#.merge(atleast_oneAIAN_race)#.merge(withdrawn)

In [ ]:
AIANs = AIAN_WD.merge(atleast_oneAIAN_race)
AIANs.participant_id.nunique()

In [ ]:
#thoose between btach 1 dates
AIANs[(AIANs.DateBloodSampleCollected > '2020-02-26 19:45:00') & (AIANs.DateBloodSampleCollected < '2020-03-18 16:04:00')]

In [ ]:
#thoose between btach 2 dates
AIANs[(AIANs.DateBloodSampleCollected > '2020-02-12 19:20:00') & (AIANs.DateBloodSampleCollected < '2020-02-26 19:43:00')]

In [ ]:
'2020-03-18 16:04:00', '2020-02-26 19:45:00')

In [ ]:
WDs = AIAN_WD.merge(withdrawn)
WDs.participant_id.nunique()
WDs[(WDs.DateBloodSampleCollected > '2020-02-26 19:45:00') & (WDs.DateBloodSampleCollected < '2020-03-18 16:04:00')]

In [ ]:
#batch 2
WDs = AIAN_WD.merge(withdrawn)
WDs.participant_id.nunique()
WDs[(WDs.DateBloodSampleCollected > '2020-02-12 19:20:00') & (WDs.DateBloodSampleCollected < '2020-02-26 19:43:00')]

# **Master List Pull** -  2020 Specimens

Batches of specimen to be provided to QUEST will be pulled from the Master list.

*`400 μL of serum will be provided to Quest from the biobank in tubes including target participant samples from 2020'*

In [ ]:
COVID_blood_samples = pd.read_sql('''

    SELECT distinct participant_id, p.biobank_id, 
    #p.hpo_id, (select name from hpo h where h.hpo_id = p.hpo_id) as hpo,
    #date_of_birth as DateOfBirth, sex_id, gender_identity_id, sexual_orientation_id,
    (select display from code where code_id = state_id) as state,
    race as Race,    
    LEFT(zip_code, 5) as ZipCode,
    bos.test, bos.collected as DateBloodSampleCollected,
    bss.created as DateBloodSampleReceived

    
    FROM participant p
    INNER JOIN participant_summary ps USING(participant_id)
    INNER JOIN biobank_order bo USING(participant_id)
    INNER JOIN biobank_ordered_sample bos on bo.biobank_order_id = bos.order_id
    INNER JOIN biobank_order_identifier boi on boi.biobank_order_id = bo.biobank_order_id
    INNER JOIN biobank_stored_sample bss on bss.biobank_order_identifier = boi.value
    
    WHERE p.participant_id NOT IN {0} ## remove all who selected at least one AIAN race
    AND {1} #exclude withdrawn, fake, tests
    AND bos.collected > 2020-01-01 00:00:00' #'2019-10-01 00:00:00'  # need to change this to '2020-01-01 00:00:00'per Mine's email on may 21, 2020
                                                #to avoid running the whole query again, the 2020 date constraint was adjust in 2.3
    AND bss.status = 1
    AND lower(bos.test) IN {2}
    
   ## group by 1, bos.test, bos.collected, bss.confirmed, bss.status
    
    '''.format(wd_aian, exclude, blood_tests2), cnx)

In [ ]:
COVID_blood_samples.participant_id.drop_duplicates().count()

## Data Quality Checks and Cleansing

### Checking whether we have instances with received date < collected date

In [ ]:
diff = COVID_blood_samples[['participant_id', 'biobank_id','DateBloodSampleReceived', 'DateBloodSampleCollected']]
diff['diff'] = pd.DataFrame(pd.to_datetime(COVID_blood_samples['DateBloodSampleReceived']) -  pd.to_datetime(COVID_blood_samples['DateBloodSampleCollected']))
diff

In [ ]:
diff[['participant_id','biobank_id','diff']].drop_duplicates().sort_values('diff')#.groupby(['diff']).count()

In [ ]:
diff.participant_id.drop_duplicates().count()

In [ ]:
COVID_blood_samples.columns

### Getting the maximum dates for Collected and Received Samples

In [ ]:
max_dates = COVID_blood_samples[['participant_id', 'biobank_id','DateBloodSampleCollected', 'DateBloodSampleReceived']].groupby(['participant_id', 'biobank_id'], as_index = False).max().drop_duplicates()

In [ ]:
max_dates

In [ ]:
COVID_blood_samples2 = pd.merge(COVID_blood_samples.drop(['DateBloodSampleCollected', 'DateBloodSampleReceived'],
                                                        axis = 1).drop_duplicates(), max_dates, on =  ['participant_id','biobank_id'])

In [ ]:
COVID_blood_samples2

In [ ]:
COVID_blood_samples2.test = [x.lower() for x in COVID_blood_samples2.test]
blood_tests.test = [x.lower() for x in blood_tests.test]

In [ ]:
COVID_blood_samples3 = pd.merge(COVID_blood_samples2, blood_tests, on = 'test', how = 'inner')
COVID_blood_samples3 = COVID_blood_samples2.drop('test', axis = 1).drop_duplicates()
COVID_blood_samples3.head()

In [ ]:
COVID_blood_samples3.shape[0],COVID_blood_samples3.participant_id.drop_duplicates().count(), COVID_blood_samples3.biobank_id.drop_duplicates().count()

## Adding code names to demographics

In [ ]:
races = pd.read_csv('../INPUTS/ps.race_codes.csv')
races.head()

In [ ]:
sex = pd.read_sql('''

    select distinct code_id as sex_id, REPLACE(short_value, 'SexAtBirth_', '') as SexAtBirth
    from code 
    where topic = 'Biological Sex'
    
    ''', cnx)

gi = pd.read_sql('''

    select distinct code_id as gender_identity_id, REPLACE(short_value, 'GenderIdentity_', '') as GenderIdentity
    from code 
    where topic = 'Gender'
    
    ''', cnx)

so = pd.read_sql('''

    select distinct code_id as sexual_orientation_id, REPLACE(short_value, 'SexualOrientation_', '') as SexualOrientation
    from code 
    where topic = 'Sexual Orientation'
    
    ''', cnx)

ethnicity = pd.read_sql('''

    SELECT distinct participant_id, 'Hispanic' as Ethnicity
    FROM participant_race_answers
    WHERE code_id = 207

     ''', cnx)

ethnicity.head()

In [ ]:
COVID_blood_samples3 = pd.merge(COVID_blood_samples2, races, how = 'left')
COVID_blood_samples3 = pd.merge(COVID_blood_samples3, gi, how = 'left')
COVID_blood_samples3 = pd.merge(COVID_blood_samples3, sex, how = 'left')
COVID_blood_samples3 = pd.merge(COVID_blood_samples3, so, how = 'left').drop(['Race', 'sexual_orientation_id',
                                                                              'sex_id','gender_identity_id'], axis =1)

COVID_blood_samples3 = pd.merge(COVID_blood_samples3, ethnicity, how = 'left')
COVID_blood_samples3['Ethnicity'] = COVID_blood_samples3['Ethnicity'].fillna('Non Hispanic')
COVID_blood_samples3[['SexualOrientation','GenderIdentity','SexAtBirth']] = COVID_blood_samples3[['SexualOrientation','GenderIdentity','SexAtBirth']].fillna('Unset')


COVID_blood_samples3.head()

In [ ]:
COVID_blood_samples4 = pd.merge(COVID_blood_samples3, blood_tests).drop_duplicates()
COVID_blood_samples4

### Concatenating Blood test types per PID so that we do not have double rows

In [ ]:
COVID_blood_samples4['fluid'] = COVID_blood_samples4['fluid']+', '

In [ ]:
types = COVID_blood_samples4[['participant_id', 'fluid']]
types = types.groupby('participant_id').sum().reset_index().rename(columns = {'fluid':'SpecimenTypes'})
#types

In [ ]:
COVID_blood_samples5 = pd.merge(COVID_blood_samples4.drop('fluid', axis = 1), types, on = 'participant_id', how = 'left').drop_duplicates()
COVID_blood_samples5.head()

In [ ]:
# Check if any column has null values
COVID_blood_samples5.isna().drop_duplicates()

In [ ]:
#COVID_blood_samples5['Ethnicity'] = COVID_blood_samples5['Ethnicity'].fillna('Unset')

In [ ]:
## rearranging column names
COVID_blood_samples_final = COVID_blood_samples5[['participant_id', 'biobank_id', 'state',
                                                      'DateOfBirth', 'SexAtBirth',
                                                      'GenderIdentity', 'SexualOrientation', 'race', 'Ethnicity', 
                                                      'ZipCode', 'DateBloodSampleCollected','DateBloodSampleReceived', 
                                                      'SpecimenTypes']]

In [ ]:
COVID_blood_samples_final

In [ ]:
## dropping 2 pids with negative diff dates that we found above 
COVID_blood_samples_final = COVID_blood_samples_final[COVID_blood_samples_final.participant_id != 537352586]
COVID_blood_samples_final = COVID_blood_samples_final[COVID_blood_samples_final.participant_id != 855613070]
COVID_blood_samples_final

### checking dates again

In [ ]:
COVID_blood_samples_final

In [ ]:
diff = COVID_blood_samples_final[['participant_id', 'biobank_id','DateBloodSampleReceived', 'DateBloodSampleCollected']]
diff['diff'] = pd.DataFrame(pd.to_datetime(COVID_blood_samples_final['DateBloodSampleReceived']) -  pd.to_datetime(COVID_blood_samples_final['DateBloodSampleCollected']))
diff

In [ ]:
diff[['participant_id','biobank_id','diff']].drop_duplicates().sort_values('diff')#.groupby(['diff']).count()

In [ ]:
COVID_blood_samples_final = COVID_blood_samples_final.sort_values('DateBloodSampleCollected', ascending= False)
COVID_blood_samples_final

In [ ]:
COVID_blood_samples_final.shape[0],COVID_blood_samples_final.participant_id.drop_duplicates().count(), COVID_blood_samples_final.biobank_id.drop_duplicates().count()

In [ ]:
COVID_blood_samples5[['participant_id','biobank_id','DateBloodSampleCollected', 'DateBloodSampleReceived']].drop_duplicates().groupby(['participant_id','biobank_id']).count().sort_values('DateBloodSampleCollected', ascending = False)

In [ ]:
print('Dates of collection and reception are unique. Some demographics are not, which is fine because those are multiselect.')

In [ ]:
#COVID_blood_samples_final.to_csv('Antobodies_tests_Master_List.csv', index = False)

In [ ]:
list(COVID_blood_samples_final.biobank_id)

## **Jan 2020** Master List Clean Up 
Keeping in Master File only those who have blood serums according to Mine's List

In [ ]:
master_list = pd.read_csv('Antobodies_tests_Master_List_05_18_2020.csv')
master_list.head()

In [ ]:
master_list_beforeJan2020 = master_list[master_list.DateBloodSampleCollected < '2020-01-01 00:00:00']
master_list_beforeJan2020.DateBloodSampleCollected.max()

In [ ]:
master_list_afterJan2020 = master_list[master_list.DateBloodSampleCollected >= '2020-01-01 00:00:00']
master_list_afterJan2020.DateBloodSampleCollected.max()

In [ ]:
master_list2.participant_id.nunique()#shape

### Checking the bids with avaiable serums in Mine's list that are in our after jan 2020 list, removing AIANs and WD

In [ ]:
batch1_SerumAvailability = pd.read_csv('Serology_batch1_SerumAvailability.csv')
batch1_SerumAvailability['biobank_id'] = [batch1_SerumAvailability.loc[x,'Subject Description'][1:] for x in batch1_SerumAvailability.index]
batch1_SerumAvailability['biobank_id'] = [int(x) for x in batch1_SerumAvailability['biobank_id']]

batch1_SerumAvailability.head()

In [ ]:
batch1_SerumAvailability.biobank_id.nunique()

In [ ]:
batch1_SerumAvailability_maybeAIANnWD = pd.merge(batch1_SerumAvailability, AIAN_races, how = 'left')
batch1_SerumAvailability_maybeAIANnWD = pd.merge(batch1_SerumAvailability_maybeAIANnWD, All_wd, how = 'left')
# batch1_SerumAvailability_AIAN = batch1_SerumAvailability_maybeAIANnWD[batch1_SerumAvailability_maybeAIANnWD.race_c == 'WhatRaceEthnicity_AIAN']
# batch1_SerumAvailability_AIAN.biobank_id.nunique()
batch1_SerumAvailability_maybeAIANnWD.head()

In [ ]:
batch1_SerumAvailability_noAIAN = batch1_SerumAvailability_maybeAIANnWD[batch1_SerumAvailability_maybeAIANnWD.race_c != 'WhatRaceEthnicity_AIAN']
batch1_SerumAvailability_noAIANnoWD = batch1_SerumAvailability_noAIAN[batch1_SerumAvailability_noAIAN.withdrawal_status != 2]


batch1_SerumAvailability_noAIANnoWD.biobank_id.nunique()

In [ ]:
batch1_SerumAvailability_noAIANnoWD.head()

In [ ]:
serum_availableafterjan2020 = pd.merge(master_list_afterJan2020, 
                                       batch1_SerumAvailability_noAIANnoWD[['biobank_id']],
                                       how = 'inner')

In [ ]:
serum_availableafterjan2020.biobank_id.nunique()#shape

In [ ]:
serum_availableafterjan2020.race.drop_duplicates()

In [ ]:
diff =  set(batch1_SerumAvailability.biobank_id) - set(serum_availableafterjan2020.biobank_id)
len(diff)

##### Merge with our before Jan list with Mine's list of available serum, no AIAN and WD

In [ ]:
master_list_beforeJan2020.head()

In [ ]:
serum_availableafterjan2020.head()

### Save Final Sample Master file

In [ ]:
#master_list_latest = master_list_beforeJan2020.append(serum_availableafterjan2020)
master_list_latest = serum_availableafterjan2020
master_list_latest= master_list_latest.drop_duplicates()
master_list_latest.participant_id.nunique()

In [ ]:
#master_list_latest.to_csv('Antobodies_tests_Master_List_'+now+'.csv', index = False)

In [ ]:
master_list = pd.read_csv('../DATA\DSC2\Antobodies_tests_Master_List_2020-05-21.csv').iloc[:,1:]

In [ ]:
master_list

## **10.22.2020** Master List Clean Up

In [ ]:
master_list_afterbatch3 = pd.read_csv('../DATA\DSC2\master_list_afterBatch3_2020-08-11_14.18.07.csv')
master_list_original = pd.read_csv('../DATA\DSC2\Antobodies_tests_Master_List_2020-05-21.csv')

master_list_afterbatch3.head(3)

### load all specimens that were used in previous batches

In [ ]:
# Load Previous batches
batch1_manifest = pd.read_csv('../DATA\\DSC2\\batch1_manifest_fromMayo.csv')#[['biobank_id']]
bath1_testsamples = transform_manifest(batch1_manifest, 'Biobank ID', master_list_df= master_list_original,
                                       which_df = 'test samples')

batch2_manifest = pd.read_csv('../DATA\\DSC2\\batch2_manifest_fromMayo.csv')#[['biobank_id']] #Manifest file from Mayo received July 22?
bath2_testsamples = transform_manifest(batch2_manifest, 'Biobank ID', master_list_df= master_list_original,
                                       which_df = 'test samples')

batch3_manifest = pd.read_csv(data_directory+'Quest_AoU_Serology_Batch3_manifest_updated.csv').rename(columns = {'Sample ID':'Specimen ID'})
batch3_manifest.loc[~batch3_manifest['Partners ID'].isnull(), 'Biobank ID'] = batch3_manifest.loc[~batch3_manifest['Partners ID'].isnull(), 'Partners ID']
batch3_manifest = batch3_manifest.drop('Partners ID', axis = 1)

bath3_testsamples = transform_manifest(batch3_manifest, 'Biobank ID', master_list_df= master_list_original,
                                       which_df = 'test samples')


In [ ]:
# Get pervious batches negative controls list
batch12_noncontrols_bids = bath1_testsamples[['biobank_id']].merge(bath2_testsamples[['biobank_id']], 'outer').drop_duplicates()
batch123_noncontrols_bids = batch12_noncontrols_bids.merge(bath3_testsamples[['biobank_id']], 'outer').drop_duplicates()#.merge(list4_negativecontrols[['biobank_id']], 'outer').drop_duplicates()
display(batch123_noncontrols_bids.head())
batch123_noncontrols_bids.biobank_id.nunique()

### load all files with specimens that have insufficient serums

In [ ]:
insufficient_serum_batch4 = pd.read_excel(data_directory+'Mayo Serology Batch Replacements Needed.xlsx')
insufficient_serum_batch4 = insufficient_serum_batch4[~insufficient_serum_batch4['Biobank ID'].str.startswith('PIO')]
insufficient_serum_batch4['biobank_id'] = [int(i.split('A')[1]) for i in insufficient_serum_batch4['Biobank ID']]

insufficient_serum_batch4_total = pd.DataFrame(['568329493'], columns = ['biobank_id']).append(insufficient_serum_batch4)[['biobank_id']]
insufficient_serum_batch4_total

In [ ]:
master_list_afterbatch3.biobank_id.nunique()

In [ ]:
master_list_afterbatch4 = pd.DataFrame(set(master_list_afterbatch3.biobank_id) - 
                                      (set(insufficient_serum_batch4_total.biobank_id).union(set(insufficient_serum_batch4_total.biobank_id))),
            columns = ['biobank_id'])


master_list_afterbatch4 = master_list_afterbatch4.merge(master_list_original).drop_duplicates()

In [ ]:
master_list_afterbatch4.participant_id.nunique()

### Save new master list

In [ ]:
save_this_csv(master_list_afterbatch4, 'master_list_afterbatch4')

------------------------------------

# **Negative Controls Master List PULL** 2019 Specimens

*'negative control participant samples date matched from 2019 ...Negative controls will be obtained from All of Us participants who enrolled in the program in 2019 matched to date range of the batch with geographic location matching as well.'*

In [ ]:
COVID_controls = pd.read_sql('''

    SELECT distinct participant_id, p.biobank_id, 
    #p.hpo_id, (select name from hpo h where h.hpo_id = p.hpo_id) as hpo,
    #date_of_birth as DateOfBirth, sex_id, gender_identity_id, sexual_orientation_id,
    race as Race, (select display from code where code_id = state_id) as state,   
    LEFT(zip_code, 5) as ZipCode,
    bos.test, bos.collected as DateBloodSampleCollected,
    bss.created as DateBloodSampleReceived

    
    FROM participant p
    INNER JOIN participant_summary ps USING(participant_id)
    INNER JOIN biobank_order bo USING(participant_id)
    INNER JOIN biobank_ordered_sample bos on bo.biobank_order_id = bos.order_id
    INNER JOIN biobank_order_identifier boi on boi.biobank_order_id = bo.biobank_order_id
    INNER JOIN biobank_stored_sample bss on bss.biobank_order_identifier = boi.value
    
    WHERE p.participant_id NOT IN {0} 
    AND participant_id NOT IN {1} ## remove all who selected at least one AIAN race
    AND {2} #exclude withdrawn, fake, tests
    AND bos.collected BETWEEN '2019-02-27 00:00:00' AND '2019-03-18 23:59:59'
    AND bss.status = 1
    AND lower(bos.test) IN {3}
    
   ## group by 1, bos.test, bos.collected, bss.confirmed, bss.status
    
    '''.format(tuple(COVID_blood_samples_final.participant_id),
               PIDS_atleast_oneAIAN_race, exclude, blood_tests2), cnx)

In [ ]:
COVID_controls.head()

In [ ]:

#COVID_controls.to_csv('COVID_controls_'+now+'.csv', index = False)

In [ ]:
#fileset[3]
COVID_controls = pd.read_csv(fileset[3]).drop_duplicates()
COVID_controls.head()

In [ ]:
COVID_controls = COVID_controls.drop('test', axis = 1).drop_duplicates()

In [ ]:
COVID_controls.participant_id.drop_duplicates()

In [ ]:
COVID_controls.biobank_id.nunique()

## Negative Control Master List Cleanup
Remove from control Master File pids with unavailable serums at the biobank

On May 20, 2020, Mine sent a file with all biobank ids that have serum available at the Mayo biobank from 2019. We need to check our control master file against that list to make sure all the controls we add tho the batches have available serum at the Mayo biobank.

### Load the file of available serum from 2019 and clean it up

In [ ]:
fileset[11]

In [ ]:
availableserums_4negcontrols = pd.read_csv(fileset[11])
availableserums_4negcontrols.head()

In [ ]:
## From my conversation with francis, Mine included some biobank IDs starting with B. These should be removed
availableserums_4negcontrols[availableserums_4negcontrols.Externalparticipant.str.startswith('B')]


In [ ]:
availableserums_4negcontrols = availableserums_4negcontrols[~availableserums_4negcontrols.Externalparticipant.str.startswith('B')]

In [ ]:
#Removing the letter 'A' in front of the biobank IDS
availableserums_4negcontrolsPids = availableserums_4negcontrols.drop_duplicates()
#availableserums_4negcontrolsPids.Externalparticipant = [int(i.split('A')[1]) for i in availableserums_4negcontrolsPids.Externalparticipant]
#availableserums_4negcontrolsPids = pd.DataFrame(availableserums_4negcontrolsPids)
availableserums_4negcontrolsPids = availableserums_4negcontrolsPids.rename(columns = {'Externalparticipant':'biobank_id'})
availableserums_4negcontrolsPids

#### removing samples with insufficient serums that were still in Mine's list - jUNE 15,2020

In [ ]:
availableserums_4negcontrolsPids = availableserums_4negcontrolsPids[availableserums_4negcontrolsPids.biobank_id != 'A538648580'].drop_duplicates()
availableserums_4negcontrolsPids

In [ ]:
len(availableserums_4negcontrolsPids)

In [ ]:
availableserums_4negcontrolsPids.to_csv('../DATA/DSC2/NegControls_mustMatch_availableSerumsin2019_'+now+'.csv', index = False)

### June 15, 2020 - Removing another unavaible serum from list

On June, 2020 Mine said a bid wih unvailable serum (538648580) made it to the list she sent us on May 20. let's remove it from the file

In [ ]:
fileset[6]

In [ ]:
availableserums_4negcontrolsPids = pd.read_csv(fileset[6])
availableserums_4negcontrolsPids.head()

In [ ]:
availableserums_4negcontrolsPids[availableserums_4negcontrolsPids.biobank_id == 538648580]


In [ ]:
availableserums_4negcontrolsPids = availableserums_4negcontrolsPids[availableserums_4negcontrolsPids.biobank_id != 538648580]
len(availableserums_4negcontrolsPids)

### Inner merge available serum file with Controls Master file
And save new control Master file List

In [ ]:
COVID_controls2 = pd.merge(availableserums_4negcontrolsPids, COVID_controls)
COVID_controls2.head()

In [ ]:
COVID_controls2.shape

## Match the Negative Control States with the Samples States

In [ ]:
COVID_controls2= pd.merge(COVID_controls2, master_list_latest[['state']]).drop_duplicates()

In [ ]:
COVID_controls2.biobank_id.nunique()

In [ ]:
COVID_controls2 = COVID_controls2[COVID_controls2.biobank_id != 538648580]
COVID_controls2.biobank_id.nunique()

## Save Negative Controls List

In [ ]:
COVID_controls2.to_csv('../DATA/DSC2/COVID_controls_'+now+'.csv', index = False)
COVID_controls2.biobank_id.nunique()

## NEW **June 30, 2020** Received New list of available serums from Sam at the biobank. 

We need to match the master negative control list with those

In [ ]:
available_serums = pd.read_csv('../DATA\\DSC2\\11.1.2019-3.18.2020 Available Serum.csv')

In [ ]:
negative_serums.head()
available_serums.head()

### Check if all pids in the master_list have available serums

In [ ]:
available_serums_bids = available_serums[available_serums['Subject Description'].str.startswith('A')]
available_serums_KIT = available_serums[~available_serums['Subject Description'].str.startswith('A')]

In [ ]:
available_serums_bids

In [ ]:
available_serums_bids['biobank_id'] = [int(i.split('A')[1]) for i in available_serums_bids['Subject Description']]

In [ ]:
pd.merge(master_list.biobank_id.drop_duplicates(),available_serums_bids.biobank_id.drop_duplicates())

In [ ]:
master_list.biobank_id.nunique(), available_serums_bids.biobank_id.nunique()

In [ ]:
available_serums_bids['Collection Date'].min(), available_serums_bids['Collection Date'].max()

**Email from Sam:**

*Hello Aymone,
Attached is an updated list of all AoU serum in storage from 11/1/2019-3/18/2020 with participants who have already been selected in batch 1 and 2 removed. I have will also include a list with those same date ranges for 2018-2019 for negative control selection in a separate email due to file size. The volume for all samples has been included so we can ensure that there is a sample with suitable volume for each participant chosen, particularly for the negative controls that require double the volume to aliquot in duplicate. Please let me know if you have any questions.*


**There are 33K pids in the list of available serums sent by the biobank (all from 2020), excluding those from batch 1 and 2. Our master file has 54k, of which 31K are in the available serum list. Since the pids fr batches 1 and 2 have already been done.I will not merge the two files. Instead for batch 3, I will make sure that the pids are in the available serums**

In [ ]:
available_serums_bids.to_csv('../DATA\\DSC2\\11.1.2019-3.18.2020 Available Serum with bids.csv')

### Check that all henagtive controls in our master list have availble serums

In [ ]:
negative_serums = pd.read_csv('../DATA\\DSC2\\11.1.2018-3.18.2019 Negative Serum with bids.csv')
negative_serums.head()

In [ ]:
#negative_serums['Subject Description'].sort_values()

In [ ]:
#negative_serums['biobank_id'] = [int(i.split('A')[1]) for i in negative_serums['Subject Description']]

In [ ]:
#negative_serums

In [ ]:
COVID_controls2 = pd.read_csv('../DATA\DSC2\COVID_controls_2020-06-15_14.14.52.csv')

In [ ]:
pd.merge(COVID_controls2.biobank_id.drop_duplicates(),negative_serums.biobank_id.drop_duplicates())

In [ ]:
COVID_controls2.biobank_id.nunique(), negative_serums.biobank_id.nunique()

In [ ]:
COVID_controls3 = pd.merge(COVID_controls2, negative_serums).drop_duplicates()
COVID_controls3

In [ ]:
COVID_controls3.to_csv('../DATA/DSC2/COVID_controls_afterbatch2'+now+'.csv', index = False)
COVID_controls3.biobank_id.nunique()

In [ ]:
negative_serums.to_csv('../DATA\\DSC2\\11.1.2018-3.18.2019 Negative Serum with bids.csv')

## **10.22.2020** Negative Control Clean up

In [ ]:
negative_controls_master4 = pd.read_csv('../DATA\\DSC2\\negative_controls_master_afterlist4_2020.10.08.csv').iloc[:,1:]
negative_controls_master_original = pd.read_csv('../DATA\DSC2\COVID_controls_2020-06-15_14.14.52.csv') #very first

#removing all the crap and dups
negative_controls_master4 = negative_controls_master4[['participant_id','biobank_id', 'Negative Control','Positive Control']].drop_duplicates().merge(negative_controls_master_original)
negative_controls_master4.head()

In [ ]:
## from Ortho/Roche list 4
insufficient_neg_ortho_roche_batch12 = pd.DataFrame([215453393, 167765452, 570857394, 215933740], columns = ['biobank_id'])
insufficient_neg_ortho_roche_batch12

In [ ]:
negative_controls_master5 = pd.DataFrame(set(negative_controls_master4.biobank_id) -set(insufficient_neg_ortho_roche_batch12.biobank_id),
            columns = ['biobank_id'])

negative_controls_master5= negative_controls_master5.merge(negative_controls_master4).drop_duplicates()
save_this_csv(negative_controls_master5, 'negative_controls_master_afterbatch4')

In [ ]:
negative_controls_master5.head()

# **Positive Control Lists** Specimens from Partners

*'and positive controls from consortia donations.'*

In [ ]:
import pandas as pd
COVID_positive_controls = pd.read_csv('../DATA/DSC2/Vanderbilt_COVID Positive Controls_05.26.20.csv')

In [ ]:
COVID_positive_controls= COVID_positive_controls.dropna()#[['Sample ID','Positive Classification','']]
COVID_positive_controls.shape

## generating random IDs for the positive controls to give to quest -- we can match later with the map

In [ ]:
COVID_positive_controls.head()

In [ ]:
import numpy as np
seed(51011)
start = 5623
stop = 40836+ len(COVID_positive_controls)

COVID_positive_controls['serology_id'] = np.random.randint(low = start,
                                                   size = len(COVID_positive_controls),
                                                   high = stop,
                                                   dtype = 'int64')

COVID_positive_controls

In [ ]:
COVID_positive_controls.serology_id.nunique()
COVID_positive_controls_final = COVID_positive_controls

## Save positive control list

In [ ]:
COVID_positive_controls_final.to_csv('../DATA/DSC2/VandiCOVIDPositiveControls_'+now+'.csv', index = False)

###### OLD QC -- check if the 5000 are in my list -- Delete?

In [ ]:
# serum_samples1 = pd.read_csv('serum_samples_list_2020-05-06 with locations.csv')

# #removing AIAN and Withdrawn PIDS
# serum_samples1 = serum_samples1[(serum_samples1.is_AIAN_participant != 1) & (serum_samples1.withdrawal_status!=2)]
# serum_samples1.head()

# serum_samples_check = serum_samples1[['biobank_id','Description','Sequence', 'Rack', 'Position in Rack']]
# first4982.biobank_id.nunique(), serum_samples_check.biobank_id.nunique()
# common_pids= pd.merge(serum_samples_check.biobank_id.drop_duplicates(), first4982.biobank_id.drop_duplicates(), 'inner')
# common_pids

## **10.22.2020** Clean Up Positive Controls

In [ ]:
# load positive controls list
vumc_positive_controls = pd.read_csv('../DATA\\DSC2\\VUMCPositive Controls Tracker_addedCals_10_08_2020.csv')
vumc_positive_controls.head(5)

In [ ]:
insufficient_serum_batch4 = pd.read_excel(data_directory+'Mayo Serology Batch Replacements Needed.xlsx')

In [ ]:
insuffificent_pos1 = insufficient_serum_batch4[insufficient_serum_batch4['Biobank ID'].str.startswith('PIO')]
insuffificent_pos1

In [ ]:
insufficient_serum_batch4[~insufficient_serum_batch4['Biobank ID'].str.startswith('A')]


In [ ]:
insuffificent_pos1['Biobank ID'].tolist()

In [ ]:
vumc_positive_controls_10_22_2020 = vumc_positive_controls[~vumc_positive_controls['Sample ID'].isin(insuffificent_pos1['Biobank ID'].tolist())]
vumc_positive_controls_10_22_2020

In [ ]:
vumc_positive_controls_10_22_2020.to_csv(data_directory+'VUMCPositive Controls Tracker_10.22.2020.csv')